# How to create a project

This tutorial demonstrates how to create a project.

## What is a project?

A project is a speos simulation container that includes parts, material properties, sensor,
sources and simulations.

In this tutorial you will learn how to create a project from scratch or from a pre-defined .speos
file.

## Prerequisites

### Perform imports

In [1]:
import os
from pathlib import Path

from ansys.speos.core import Project, Speos
from ansys.speos.core.kernel.client import (
    default_docker_channel,
)
from ansys.speos.core.launcher import launch_local_speos_rpc_server
from ansys.speos.core.sensor import SensorIrradiance
from ansys.speos.core.simulation import SimulationDirect
from ansys.speos.core.source import SourceLuminaire, SourceSurface


### Define constants

The constants help ensure consistency and avoid repetition throughout the example.

In [2]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.

## Model Setup

### Load assets
The assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [3]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

### Start/Connect to Speos RPC Server
This Python client connects to a server where the Speos engine
is running as a service. In this example, the server and
client are the same machine. the launch_local_speos_rpc_method can
be used to start a local instance of the service.

In [4]:
if USE_DOCKER:
    speos = Speos(channel=default_docker_channel())
else:
    speos = launch_local_speos_rpc_server(port=GRPC_PORT)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/tools/common/cyberchannel.py:188: UserWarning: Starting gRPC client without TLS on localhost:50098. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


## New empty project

An empty project can be created by only passing speos rpc server to
the Project class.

In [5]:
p = Project(speos=speos)
print(p)

{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


### Create features
The Project class has a multitude of method to create Speos features.
each create methedo takes the name and the Feature type as arguments
and returns the created Feature
#### Source

In [6]:
source1 = p.create_source(name="Source.1", feature_type=SourceLuminaire)
source1.set_intensity_file_uri(uri=str(assets_data_path / "IES_C_DETECTOR.ies"))
source1.commit()

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:230: UserWarning: The pySpeos feature : SourceLuminaire needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SourceLuminaire(


#### Sensor

In [7]:
sensor1 = p.create_sensor(name="Sensor.1")
sensor1.commit()

#### Optical property

In [8]:
opt_prop1 = p.create_optical_property(name="Material.1")
opt_prop1.commit()

## Read Project

User can read the content of a project via simply printing the project

In [9]:
print(p)

{
    "sources": [
        {
            "name": "Source.1",
            "metadata": {
                "UniqueId": "b9e18564-dc32-4f95-addf-954198493ae3"
            },
            "source_guid": "e452953e-89ac-484f-8d34-46fd94e98288",
            "description": "",
            "source": {
                "name": "Source.1",
                "luminaire": {
                    "flux_from_intensity_file": {},
                    "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
                    "spectrum_guid": "4981b2cd-8888-425f-b501-d506a580a5a9",
                    "spectrum": {
                        "name": "Source.1.Spectrum",
                        "predefined": {
                            "incandescent": {}
                        },
                        "description": "",
                        "metadata": {}
                    },
                    "axis_system": [
                        0.0,
                        0.0,
                        0.0,
       

Or, user can use the find_key method to read a specific feature:

In [10]:
for it in p.find_key(key="monochromatic"):
    print(it)

## Find a feature inside a project

### Use find method with an exact name

If no feature is found, an empty list is returned.

In [11]:
features = p.find(name="UnexistingName")
print(features)

[]


In [12]:
features = p.find(name="Sensor.1")
print(features[0])

{
    "name": "Sensor.1",
    "metadata": {
        "UniqueId": "2ffa8677-4855-4ce2-957c-cc5b0e616b8f"
    },
    "sensor_guid": "bd7ec90e-9a16-489a-8c57-4e1e428a7f34",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "irradiance_sensor_template": {
            "sensor_type_photometric": {},
            "illuminance_type_planar": {},
            "dimensions": {
                "x_start": -50.0,
                "x_end": 50.0,
                "x_sampling": 100,
                "y_start": -50.0,
                "y_end": 50.0,
                "y_sampling": 100
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ],
            "layer_type_none": {},
            "ray_file_type": "RayFileNone",
            "integrati

### Use find method with feature type

Here a wrong type is given: no source is called Sensor.1 in the project

In [13]:
features = p.find(name="Sensor.1", feature_type=SourceLuminaire)
print(features)

[]


In [14]:
features = p.find(name="Sensor.1", feature_type=SensorIrradiance)
print(features[0])

{
    "name": "Sensor.1",
    "metadata": {
        "UniqueId": "2ffa8677-4855-4ce2-957c-cc5b0e616b8f"
    },
    "sensor_guid": "bd7ec90e-9a16-489a-8c57-4e1e428a7f34",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "irradiance_sensor_template": {
            "sensor_type_photometric": {},
            "illuminance_type_planar": {},
            "dimensions": {
                "x_start": -50.0,
                "x_end": 50.0,
                "x_sampling": 100,
                "y_start": -50.0,
                "y_end": 50.0,
                "y_sampling": 100
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ],
            "layer_type_none": {},
            "ray_file_type": "RayFileNone",
            "integrati

### Use find method with approximation name with regex

find a feature with name starting with Mat

In [15]:
features = p.find(name="Mat.*", name_regex=True)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)

<class 'ansys.speos.core.opt_prop.OptProp'> : name=Material.1


find all features without defining any name

In [16]:
features = p.find(name=".*", name_regex=True)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)

<class 'ansys.speos.core.source.SourceLuminaire'> : name=Source.1
<class 'ansys.speos.core.sensor.SensorIrradiance'> : name=Sensor.1
<class 'ansys.speos.core.opt_prop.OptProp'> : name=Material.1


## Delete

This erases the scene content in server database.

This deletes also each feature of the project

In [17]:
p.delete()
print(p)

{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


As the features were deleted just above -> this returns an empty vector

In [18]:
print(p.find(name="Sensor.1"))

[]


## Create project from pre-defined speos project

Via passing the .speos/.sv5 file path to the Project class.

In [19]:
p2 = Project(
    speos=speos,
    path=str(assets_data_path / "LG_50M_Colorimetric_short.sv5" / "LG_50M_Colorimetric_short.sv5"),
)
print(p2)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:811: UserWarning: The pySpeos feature : FaceStub.read_batch needs a Speos Version of 2025 R2 SP0 or higher.
  f_data_list = face_db.read_batch(refs=f_links)


{
    "name": "LG_50M_Colorimetric_short",
    "description": "From /app/assets/LG_50M_Colorimetric_short.sv5/LG_50M_Colorimetric_short.sv5",
    "part_guid": "eb2d66ae-860a-4abd-90a4-5aa944fc427c",
    "sources": [
        {
            "name": "Dom Source 2 (0) in SOURCE2",
            "metadata": {
                "UniqueId": "216c13d6-c389-4215-b1ec-5f7d496a3469"
            },
            "source_guid": "5121adce-3d6f-4a8a-97a5-8cdc2b90842f",
            "description": "",
            "source": {
                "name": "Dom Source 2 (0) in SOURCE2",
                "surface": {
                    "radiant_flux": {
                        "radiant_value": 6.590041607465698
                    },
                    "intensity_guid": "339afa82-8590-4236-925c-0763c0fabf19",
                    "exitance_constant": {
                        "geo_paths": [
                            {
                                "geo_path": "Solid Body in SOURCE2:2920204960/Face in SOURCE2:222",

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:917: UserWarning: The pySpeos feature : SourceSurface needs a Speos Version of 2025 R2 SP0 or higher.
  src_feat = SourceSurface(
/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:989: UserWarning: The pySpeos feature : SimulationDirect needs a Speos Version of 2025 R2 SP0 or higher.
  sim_feat = SimulationDirect(


## Preview the part information

User can check the project part using preview method.

In [20]:
p2.preview()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html lang=&quot;en-US&quot; dir=&quot;ltr&quot;>\n  …

use find_key method to find specific information

In [21]:
for it in p2.find_key(key="surface"):
    print(it)

(".sources[.name='Dom Source 2 (0) in SOURCE2'].source.surface", {'radiant_flux': {'radiant_value': 6.590041607465698}, 'intensity_guid': '339afa82-8590-4236-925c-0763c0fabf19', 'exitance_constant': {'geo_paths': [{'geo_path': 'Solid Body in SOURCE2:2920204960/Face in SOURCE2:222', 'reverse_normal': False}]}, 'spectrum_guid': 'be7d8a26-3dc1-4d58-b9cd-a2a67e709565', 'intensity': {'cos': {'N': 1.0, 'total_angle': 180.0}, 'name': '', 'description': '', 'metadata': {}}, 'spectrum': {'library': {'file_uri': '/app/assets/LG_50M_Colorimetric_short.sv5/Red Spectrum.spectrum'}, 'name': '', 'description': '', 'metadata': {}}})
(".sources[.name='Surface Source (0) in SOURCE1'].source.surface", {'radiant_flux': {'radiant_value': 9.290411220389682}, 'intensity_guid': 'a115672d-1be5-48d5-b395-3a6d5f45d46f', 'exitance_constant': {'geo_paths': [{'geo_path': 'Solid Body in SOURCE1:2494956811/Face in SOURCE1:187', 'reverse_normal': False}]}, 'spectrum_guid': 'c17ee575-18c5-4a77-bd97-e1bd8b409e04', 'inte

Use find method to retrieve feature:

e.g. surface source

In [22]:
features = p2.find(name=".*", name_regex=True, feature_type=SourceSurface)
print(features)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)
src = features[1]

[<ansys.speos.core.source.SourceSurface object at 0x7f6e46b2fb60>, <ansys.speos.core.source.SourceSurface object at 0x7f6e3cc51090>]
<class 'ansys.speos.core.source.SourceSurface'> : name=Dom Source 2 (0) in SOURCE2
<class 'ansys.speos.core.source.SourceSurface'> : name=Surface Source (0) in SOURCE1


modify the surface source, e.g. surface source wavelength:

In [23]:
src.set_spectrum().set_monochromatic(wavelength=550)
src.commit()

Retrieve a simulation feature:

In [24]:
features = p2.find(name=".*", name_regex=True, feature_type=SimulationDirect)
sim_feat = features[0]
print(sim_feat)

{
    "name": "ASSEMBLY1.DS (0)",
    "metadata": {
        "UniqueId": "63cfa8ef-203c-443c-a13f-51e6c037dd20"
    },
    "simulation_guid": "4037c821-df11-44b7-9ec9-8cb19b8df706",
    "sensor_paths": [
        "Dom Irradiance Sensor (0)"
    ],
    "source_paths": [
        "Dom Source 2 (0) in SOURCE2",
        "Surface Source (0) in SOURCE1"
    ],
    "description": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.05,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "ambient_material_uri": ""
        },
        "name": "ASSEMBLY1.DS (0)",
        "metadata": {},
        "description": "",
        "scene_guid": "2e055aed-80f4-4c8d-9315-e3182cd438da",
        "simulation_path": "ASSEMBLY1.DS (0)",
        "job_type

In [25]:
sim_feat.compute_CPU()

[upload_response {
  info {
    uri: "5e5235a5-15d7-49ef-88cb-331728ad4dec"
    file_name: "ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp"
    file_size: 1477363
  }
  upload_duration {
    nanos: 1975559
  }
}
, upload_response {
  info {
    uri: "d9d2e5cf-5651-4c24-a954-e1e058f35e0d"
    file_name: "ASSEMBLY1.DS (0).html"
    file_size: 494756
  }
  upload_duration {
    nanos: 829068
  }
}
]

Preview simulation result (only windows)

In [26]:
if os.name == "nt":
    from ansys.speos.core.workflow.open_result import open_result_image

    open_result_image(
        simulation_feature=sim_feat,
        result_name="ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp",
    )

In [27]:
speos.close()

True